In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import time
import seaborn as sns
from tqdm import tqdm
from matplotlib.patches import Patch
import os
import pygame
import wandb

In [2]:
# Set device for training (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the QNet class
class REINFORCE(nn.Module):
    def __init__(self, env, lr=0.005, device=device):
        super(REINFORCE, self).__init__()
        
        # Set device for training (GPU if available)
        self.device = device
        
        # Get state and action space dimensions
        self.state_space_dim = env.observation_space.shape[0]
        self.action_space_dim = env.action_space.n
        
        # Define possible actions
        self.actions = torch.arange(self.action_space_dim).to(device)
        
        # Set learning rate
        self.lr = lr
        
        # Define neural network architecture
        self.net = nn.Sequential(
            nn.Linear(self.state_space_dim, 16, bias=True),
            nn.Tanh(),
            nn.Linear(16, 32, bias=True),
            nn.Tanh(),
            nn.Linear(32, self.action_space_dim, bias=True),
            nn.Softmax(dim=-1),
            )
        # Define optimizer
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.lr)

        # Define lr scheduler
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=100, gamma=0.95)
    
    def forward(self, x):
        # Forward pass through the network
        return self.net(x.to(self.device))
    
    
    # Choose action based on epsilon-greedy policy
    def act(self, state):
        '''
        Description:
                     This function greedily chooses an action based on the policy
                     network's output.

        Input:  
               state - current state of the environment
        
        Output:
                action - action to take
        '''
        # Get action probabilities
        action_probs = self.forward(state)
              
        #torch multinomial returns a tensor of indices of length 1, 
        #choosing the action with the highest probability as the action to take

        action = torch.multinomial(action_probs, 1).item()  
        
        return action
        
    def update_policy(self, rewards, log_probs, gamma):
        '''
        Description: 
                     This function calculates the policy gradient loss for an episode
                     and updates the model's weights. It uses the collected rewards and
                     the log probabilities of the actions taken to adjust the policy in
                     a direction that increases the expected return.
                        
        Input: 
               rewards - list of rewards collected during the episode
        
               log_probs - list of log probabilities corresponding to 
                           the probability of taking each action that
                           was actually taken, as computed by the policy network.
                           
               gamma - discount factor which is used to calculate the present value of
                       future rewards, thereby weighing immediate rewards against future
                       rewards
               
        Output: 
                loss - policy gradient loss
        '''
        
        # Compute discounted rewards going backwards from the last state of the episode
        discounted_rewards = []
        R = 0
        
        for r in rewards[::-1]:    
            # Reset reward sum if we encounter a non-zero reward
  
            R = r + gamma * R
            
            # Insert discounted reward at the beginning of the list
            discounted_rewards.insert(0, R)
    
        # Convert discounted rewards to tensor and pass to device
        discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32).to(self.device)
        
        #Normalize discounted rewards
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        #List to store policy gradient loss 
        policy_gradient_loss = []
        
        # Compute policy gradient loss for each step of the episode
        for log_prob, reward in zip(log_probs, discounted_rewards):
            
            # append the negative of the policy gradient loss to the list
            policy_gradient_loss.append(-log_prob * reward)
        
        #Stack the sum of the policy gradient losses of this episode to the list of policy gradient loss    
        loss = torch.stack(policy_gradient_loss).sum()
        
        # Reset gradients to zero
        self.optimizer.zero_grad() 
        
        # Backpropagate the loss
        loss.backward()
        
        # Perform the optimization step
        self.optimizer.step()
        
        return loss.item() # Return the loss as a float for logging
        
    def save_model(self, filename, path):
        # Save model to file in path
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), path + filename)
        
    
    def load_model(self, filename, device='cuda'):
        # Load model from file
        self.load_state_dict(torch.load(filename, map_location=device))
        


In [3]:
#Auxiliary functions


# Function to preprocess states
def preprocess_state(state):
    '''
    This function converts the state to a tensor and adds a batch
    dimension for PyTorch to process.
    '''
    # Convert state to a numpy array
    return torch.from_numpy(state).float().unsqueeze(0).to(device)

# Function to clear the video directory
def log_clear_video_directory(video_dir):
    '''
    This function clears the video directory of all video files
    and logs the most recent video file to wandb.
    '''  
    # Get list of video files in video directory 
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    
    # After each episode, log the video file to wandb
    if video_files:
        last_video_file = video_files[-1]  # Assuming the last file in the list is the most recent
        wandb.log({"episode_video": wandb.Video(os.path.join(video_dir, last_video_file), fps=4, format="mp4")})
    
    # Delete all video files in video directory
    for file in video_files:
        os.remove(os.path.join(video_dir, file))
  
# Function to log the episode rewards
def log_rewards(episode_reward, last100_avg_reward, full_avg_reward):
    # Log episode rewards
    wandb.log({'Episode reward': episode_reward, 'Lat 100 episodes Average reward': last100_avg_reward, "Full average reward": full_avg_reward})

In [4]:

# Initialize Weights & Biases
wandb.init(project="v3_reinforce_lunar_lander", entity="ai42")


env = gym.make('LunarLander-v2', continuous=False, render_mode='rgb_array')
REINFORCE = REINFORCE(env).to(device)
optimizer = optim.Adam(REINFORCE.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)

#HYPERPARAMETERS
lr=0.005 # learning rate
gamma= 0.99 # discount factor
batch_size= 128 # batch size
max_episodes= 5000 # maximum number of episodes


# Define a video directory
video_dir = '/home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/' # Set this to your preferred directory
os.makedirs(video_dir, exist_ok=True)

# Wrap your environment to record videos
env = gym.wrappers.RecordVideo(env, video_folder=video_dir, episode_trigger=lambda episode_id: True)

# Best score initialization
best_score = -np.inf

# Set up lists to hold results
all_rewards = []
avg_rewards = []
losses = []

# Replace 'max_episodes' with the actual number
for episode in range(max_episodes):
    
    # Reset environment and record the starting state
    state, info = env.reset()
    
    # Preprocess state
    state = preprocess_state(state)

    # Initialize lists to store rewards and log probabilities
    log_probs = []
    rewards = []
    
    # Initialize total reward for episode
    total_reward = 0
    
    # Run episode
    for t in range(env.spec.max_episode_steps):  # Limit the number of timesteps per episode:  env.spec.max_episode_steps by default is 1000
        # Get action from agent 
        action = REINFORCE.act(state)
        
        # Take step in environment 
        next_state, reward, terminated, truncated, info = env.step(action)
        
        # Preprocess next state
        next_state = preprocess_state(next_state)

         #we use the log_prob method of the distribution to get the log probability of the action taken in the current state 
        log_prob = torch.log(REINFORCE.forward(state)[0][action])
        
        # Append log_prob to log probs list and reward to rewards list
        log_probs.append(log_prob)
        rewards.append(reward)

        # Update total reward
        total_reward += reward
        
        # Update state
        state = next_state

        #Break if terminated or truncated
        if terminated or truncated:
            break
     
    # Calculate loss using the reinforce_agent.update_policy() method 
    loss = REINFORCE.update_policy(rewards, log_probs, gamma)
    
    # Append loss to losses list
    losses.append(loss)

    # append the total reward of the episode to the list of all rewards
    all_rewards.append(total_reward)
    
    # Calculate average reward of the last 100 episodes and save to variable
    last100_avg_reward = np.mean(all_rewards[-100:])  # Average reward of the last 100 episodes
    
    # Calculate average reward of all episodes and save to variable
    full_avg_reward = np.mean(all_rewards)  # Average reward of all episodes
    
    # Log total_reward of episode, last 100 episodes average reward, and full historic average reward to wandb
    log_rewards(total_reward, last100_avg_reward, full_avg_reward)
    
    # Log loss to wandb
    wandb.log({"loss": loss})

    #Log and clear video directory
    log_clear_video_directory(video_dir)
    
    #Save model if best score
    if total_reward > best_score:
        best_score = total_reward
        REINFORCE.save_model('reinforce_lunar_lander_best_v3.pth', path='/home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce/weights')

    wandb.log({"best_score": best_score})
    wandb.log({"episode": episode})
    
    #scheduler step for learning rate decay
    scheduler.step()
    
# Close the environment and finish wandb run
env.close()
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: neildlf (ai42). Use `wandb login --relogin` to force relogin


/home/ndelafuente/miniconda3/envs/pytorch-env/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-0.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-0.mp4


/home/ndelafuente/miniconda3/envs/pytorch-env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-318.mp4
Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-319.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-319.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-319.mp4


Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-320.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-320.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-320.mp4
Moviepy - Building video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-321.mp4.
Moviepy - Writing video /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-321.mp4



Moviepy - Done !
Moviepy - video ready /home/ndelafuente/Desktop/Learn2Earn_RL/LunarLander/Reinforce_Lunar_Lander/Videosv3/rl-video-episode-321.mp4
